<a href="https://colab.research.google.com/github/harveenchadha/bol/blob/main/demos/gr/hindi/gr_vakyansh_hindi_him_4200.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio
!pip install transformers torchaudio
!apt-get install sox

     |████████████████████████████████| 7.4 MB 4.5 MB/s 
     |████████████████████████████████| 206 kB 42.3 MB/s 
     |████████████████████████████████| 1.9 MB 38.0 MB/s 
     |████████████████████████████████| 961 kB 43.0 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
     |████████████████████████████████| 3.2 MB 36.3 MB/s 
  Created wheel for ffmpy: filename=ffmpy-0.3.0-py3-none-any.whl size=4709 sha256=545eb72c4d69d5c07894145f7130d334c8bd15e5e045b5c04bb82f94de4c5bfe
  Stored in directory: /root/.cache/pip/wheels/13/e4/6c/e8059816e86796a597c6e6b0d4c880630f51a1fcfa0befd5e6
  Created wheel for flask-cachebuster: filename=Flask_CacheBuster-1.0.0-py3-none-any.whl size=3372 sha256=5fc5e093fcbbdeb311f18ab222bf7a3dadfa914b0e946a35680ad9415d6a6f1b
  Stored in directory: /root/.cache/pip/wheels/28/c0/c4/44687421dab41455be93112bd1b0dee1f3c5a9aa27bee63708
Successfully built ffmpy flask-cachebuster
     |████████████████████████████████| 2.6 MB 5.0 MB/s 
     |█████████████████

In [74]:
import soundfile as sf
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import argparse
from glob import glob
import torchaudio
import subprocess
import gradio as gr

resampler = torchaudio.transforms.Resample(48_000, 16_000)

def get_filename(wav_file):
    filename_local = wav_file.split('/')[-1][:-4]
    filename_new = '/tmp/'+filename_local+'_16.wav'
    
    
    subprocess.call(["sox {} -r {} -b 16 -c 1 {}".format(wav_file, str(16000), filename_new)], shell=True)
    return filename_new

def parse_transcription(wav_file):
    # load pretrained model
    processor = Wav2Vec2Processor.from_pretrained("Harveenchadha/vakyansh-wav2vec2-hindi-him-4200")
    model = Wav2Vec2ForCTC.from_pretrained("Harveenchadha/vakyansh-wav2vec2-hindi-him-4200")

    # load audio

    
    wav_file = get_filename(wav_file.name)
    audio_input, sample_rate = sf.read(wav_file)
    #test_file = resampler(test_file[0])

    # pad input values and return pt tensor
    input_values = processor(audio_input, sampling_rate=16_000, return_tensors="pt").input_values

    # INFERENCE
    # retrieve logits & take argmax
    logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)

    # transcribe
    transcription = processor.decode(predicted_ids[0], skip_special_tokens=True)
    return transcription

In [71]:
input = gr.inputs.Audio(source="microphone", type="file") 
gr.Interface(parse_transcription, inputs = input,  outputs="text", 
             analytics_enabled=False, show_tips=False).launch(inline=False);

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Your interface requires microphone or webcam permissions - this may cause issues in Colab. Use the External URL in case of issues.
This share link will expire in 24 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted (NEW!)
Running on External URL: https://21100.gradio.app


KeyboardInterrupt: ignored